In [1]:
from huggingface_hub import hf_hub_download
from ultralytics import YOLO
from supervision import Detections
from PIL import Image
from PIL import ImageOps

c:\Users\yango\anaconda3\envs\eunseo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#추가 임포트
import os

In [3]:
model_path = hf_hub_download(repo_id="arnabdhar/YOLOv8-Face-Detection", filename="model.pt")
# load model
model = YOLO(model_path)

In [ ]:

base_dir = os.path.dirname(os.path.abspath(__file__))
input_root = os.path.join(base_dir,'Data','img')

#결과 저장 폴더 path
output_root= os.path.join(base_dir,'CropData2','img')

valid_extensions =(".jpg", ".jpeg", ".png") #안전장치
    

In [5]:
for root, _, files in os.walk(input_root) :
    for filename in files :
        if filename.lower().endswith(valid_extensions) :
            image_path = os.path.join(root,filename)
            image = Image.open(image_path)
            image = ImageOps.exif_transpose(image).convert('RGB')  # 회전 보정
            
            #모델 추론~
            output = model(image)
            detections = Detections.from_ultralytics(output[0])
            
            rel_path = os.path.relpath(root, input_root)
            save_dir = os.path.join(output_root, rel_path)
            os.makedirs(save_dir, exist_ok=True)
            
            for i, (box, conf) in enumerate(zip(detections.xyxy, detections.confidence)) :
                if conf < 0.7:  # 신뢰도 낮은 탐지는 무시
                    continue
                x1,y1,x2,y2 = map(int, box)
                face_crop = image.crop((x1,y1,x2,y2)).resize((224,224))
                
                save_name = f'{os.path.splitext(filename)[0]}_crop_2{i}.jpg'
                save_path = os.path.join(save_dir, save_name)
                face_crop.save(save_path)


0: 480x640 2 FACEs, 61.2ms
Speed: 2.9ms preprocess, 61.2ms inference, 61.6ms postprocess per image at shape (1, 3, 480, 640)

0: 320x640 1 FACE, 60.0ms
Speed: 1.3ms preprocess, 60.0ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 480x640 1 FACE, 8.3ms
Speed: 2.0ms preprocess, 8.3ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 FACE, 5.5ms
Speed: 1.7ms preprocess, 5.5ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 FACEs, 5.9ms
Speed: 2.8ms preprocess, 5.9ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 512x640 1 FACE, 69.0ms
Speed: 1.9ms preprocess, 69.0ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)

0: 480x640 1 FACE, 6.5ms
Speed: 2.2ms preprocess, 6.5ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 512x640 1 FACE, 6.3ms
Speed: 2.0ms preprocess, 6.3ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)

0: 480x640 1 F